![image](https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/notebooks/headers/watsonx-Prompt_Lab-Notebook.png)
# Prompt Notebook with Chat - Prompt Lab Notebook v1.1.0
This notebook contains steps and code to demonstrate inferencing of prompts
generated in Prompt Lab in watsonx.ai with a chat format. It introduces Python API commands
for authentication using API key and prompt inferencing using WML API.

**Note:** Notebook code generated using Prompt Lab will execute successfully.
If code is modified or reordered, there is no guarantee it will successfully execute.
For details, see: <a href="/docs/content/wsj/analyze-data/fm-prompt-save.html?context=wx" target="_blank">Saving your work in Prompt Lab as a notebook.</a>

Some familiarity with Python is helpful. This notebook uses Python 3.10.

## Notebook goals
The learning goals of this notebook are:

* Defining a Python function for obtaining credentials from the IBM Cloud personal API key
* Defining parameters of the Model object
* Using the Model object to generate response using the defined model id, parameters and the prompt input

# Setup

## watsonx API connection
This cell defines the credentials required to work with watsonx API for Foundation
Model inferencing.

**Action:** Provide the IBM Cloud personal API key. For details, see
<a href="https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui" target="_blank">documentation</a>.


In [20]:
import os
import getpass

def get_credentials():
	return {
		"url" : "https://eu-de.ml.cloud.ibm.com",
		"apikey" : getpass.getpass("Please enter your api key (hit enter): ")
	}


# Inferencing
This cell demonstrated how we can use the model object as well as the created access token
to pair it with parameters and input string to obtain
the response from the the selected foundation model.

## Defining the model id
We need to specify model id that will be used for inferencing:


In [21]:
model_id = "sdaia/allam-1-13b-instruct"


## Defining the model parameters
We need to provide a set of model parameters that will influence the
result:

In [22]:
parameters = {
    "decoding_method": "greedy",
    "max_new_tokens": 900,
    "repetition_penalty": 1
}

## Defining the project id or space id
The API requires project id or space id that provides the context for the call. We will obtain
the id from the project or space in which this notebook runs:

In [23]:
project_id = os.getenv("PROJECT_ID")
space_id = os.getenv("SPACE_ID")


## Defining the Model object
We need to define the Model object using the properties we defined so far:


In [24]:
from ibm_watsonx_ai.foundation_models import Model

model = Model(
	model_id = model_id,
	params = parameters,
	credentials = get_credentials(),
	project_id = project_id,
	space_id = space_id
	)


Please enter your api key (hit enter): ········


Error getting IAM Token.
Reason: <Response [400]>


WMLClientError: Error getting IAM Token.
Reason: <Response [400]>

## Defining the inferencing input for chat
Foundation models supporting chat accept a system prompt that instructs the model on how to conduct the dialog. They also accept previous questions and answers to give additional context when inferencing. Each model has it's own string format for constructing the input.

Let us provide the input we got from the Prompt Lab and format it for the selected model:


In [9]:
prompt_input = """<<SYS>>
المطلوب: أريدك أن تقوم بتقييم القصة المقدمة بناءً على المعايير التالية، مع التركيز على **ملاءمتها للأطفال بعمر 3-5 سنوات**، وإعطاء درجة لكل معيار، ثم احتساب المجموع النهائي. تأكد أن القصة خالية من أي محتوى غير مناسب للأطفال، وأن تكون المصطلحات بسيطة وسهلة الفهم. يجب أن تكون التقييمات دقيقة بما يتناسب مع قدرات الأطفال على الفهم في هذه الفئة العمرية، وأن يتم تقديم ملخص نهائي عن مدى ملاءمة القصة للأطفال من حيث اللغة والمضمون.

**إرشادات إضافية للتقييم:**

1. **الكلمات والمصطلحات**: إذا وُجدت كلمات قد تكون غير مألوفة أو صعبة على الأطفال بعمر 3-5 سنوات، مثل "الدروب القاحلة" أو "المروءة"، يجب خفض التقييم لهذا المعيار. يجب استخدام كلمات بسيطة ومتداولة وسهلة الفهم، مع تجنب المفردات غير الملائمة لبيئة الأطفال. إذا وُجدت أكثر من كلمة غير مألوفة، يجب خفض التقييم بشكل أكبر.

2. **التكرار والتوضيح**: التكرار يساعد الأطفال على استيعاب الفكرة والمغزى، فإذا كانت القصة تفتقر إلى تكرار الأفكار أو الجمل الأساسية، خفّض التقييم في معيار التكرار. كما يُفضّل إعادة ذكر المغزى بطرق مبسطة لضمان فهم الأطفال له.

3. **قابلية الكلمات للتصور**: يجب أن تكون الأحداث والأوصاف سهلة التخيل، فإذا تضمنت القصة بيئات أو أوصافًا قد تكون غير مألوفة أو صعبة على الطفل تصورها، مثل الصحراء القاحلة أو العادات البدوية، خفّض التقييم لمعيار قابلية الكلمات للتصور. استخدم أوصافًا بسيطة ومألوفة للأطفال، حيث إنهم يحتاجون إلى تصور الأحداث بسهولة.

4. **تسلسل الأحداث**: يجب أن يكون تسلسل الأحداث بسيطًا وواضحًا، بحيث يمكن للطفل متابعته دون مساعدة. إذا كانت القصة تحتوي على تعقيد في تسلسل الأحداث، خفّض التقييم في معيار هيكل النص. كلما زادت الحاجة إلى تفسير إضافي، يجب خفض التقييم بشكل أكبر.

5. **الملاءمة العاطفية والقيم الأخلاقية**: إذا تضمنت القصة قيمًا أو مشاعر تتطلب تفسيرًا معقدًا (مثل الشجاعة في مواقف قد تكون غير مألوفة أو تحمل صراعًا كبيرًا)، يجب خفض التقييم لمعيار الملاءمة العاطفية والقيم الثقافية. يُفضّل تقديم قيم بسيطة وواضحة ومناسبة لعمر الأطفال.

**نص القصة**:
[الرجاء إدخال نص القصة هنا]

**المعايير لتقييم القصة**:
1. **ملاءمة المحتوى للأطفال (3-5 سنوات)**: يجب أن تخلو القصة تمامًا من أي مشاهد غير مناسبة، كلمات قد تخيف الأطفال، أو مواضيع غير أخلاقية. كما يجب تجنب مواضيع العنف أو الأحداث التي لا تناسب هذه الفئة العمرية.

2. **اللغة والمصطلحات**: يجب أن تكون الكلمات بسيطة وواضحة، بحيث يسهل على الطفل فهمها دون مساعدة إضافية. يجب أن تكون المصطلحات مناسبة للأطفال بعمر 3-5 سنوات، ويفضل استخدام كلمات مألوفة تتناسب مع بيئتهم وثقافتهم.

3. **هيكل النص**: يجب أن تكون القصة ذات بنية واضحة، تحتوي على بداية ووسط ونهاية. يجب أن تكون الأحداث متتابعة بشكل يسهل فهمه دون إرباك الطفل، بحيث يشعر الطفل بتسلسل منطقي للقصة.

4. **الشخصيات**: يجب أن تكون الشخصيات واضحة، بسيطة، وقريبة من بيئة الطفل. يُفضل أن تكون الشخصيات محببة أو ترتبط بالواقع، مثل الحيوانات أو شخصيات من العائلة، وأن تكون متوافقة مع البيئة الثقافية والدينية للطفل.

5. **الخيال والواقعية**: يجب أن توازن القصة بين الخيال المعتدل والواقعية بحيث لا تثير مخاوف الطفل، لكن تبقي على عنصر التشويق. يُفضل أن يكون الخيال لطيفًا ويسهل على الطفل تخيله.

6. **التكرار في الأحداث**: يجب أن تحتوي القصة على تكرار بسيط في الأحداث أو الجمل الأساسية. فالتكرار يساعد الأطفال في هذا العمر على الفهم واستيعاب المغزى من القصة.

7. **الطول والانتباه**: يجب أن تكون القصة قصيرة كفاية لتحافظ على انتباه الطفل من البداية إلى النهاية، دون أن يشعر الطفل بالملل. إذا كانت القصة تحتوي على فقرات طويلة، يجب تجزئتها إلى أجزاء صغيرة.

8. **القيم الثقافية والدينية**: يجب أن تعزز القصة القيم الإسلامية والعربية بشكل مناسب للفئة العمرية، مع التركيز على الأخلاق مثل الكرم، التعاون، أو الصدق، دون تقديم موضوعات معقدة.

9. **قابلية الكلمات للتصور**: يجب أن تكون الكلمات والأحداث بسيطة وسهلة التصور، بحيث يتمكن الطفل من تخيل المواقف بسهولة. يُفضل استخدام أوصاف بسيطة للأماكن والشخصيات، مما يجعل القصة أقرب إلى خيال الطفل.

10. **الملاءمة العاطفية**: يجب أن تتضمن القصة مشاعر وأحداث تتناسب مع قدرة الطفل على الفهم والتعامل مع النص، بحيث لا تتضمن لحظات حزينة أو مخيفة قد تؤثر سلباً على مشاعر الطفل.

**نموذج الرد المطلوب**:
لكل معيار درجة من 1 إلى 5، مع تعليق مختصر لكل معيار يوضح الأسباب التي تؤيد الدرجة الممنوحة، مع إضافة الدرجة النهائية في النهاية.

**التقييم لكل معيار**:

1. **ملاءمة المحتوى للأطفال**: 
   - درجة: 2/5 - [يفضل الابتعاد عن موضوعات مثل [أذكر أي مواضيع غير ملائمة إن وجدت] بسبب عدم ملاءمتها لهذه الفئة العمرية]

2. **اللغة والمصطلحات**: 
   - درجة: 4/5 - [يجب أن تكون بعض المصطلحات أبسط بحيث يسهل فهمها للأطفال دون مساعدة]

3. **التكرار في الأحداث**:
   - درجة: 4/5 - [التكرار يساعد الطفل على استيعاب المغزى، يمكن تعزيز بعض الأجزاء بتكرار إضافي لجعل القصة أكثر جذباً]

...(إكمال باقي المعايير)

** مثلا 30  من 50 المجموع النهائي**: [المجموع الكلي من 50]

**تعليق نهائي مختصر**: يُنصح بتعديل بعض المصطلحات والأحداث لتصبح القصة أكثر ملاءمة للأطفال بعمر 3-5 سنوات. استخدم تكرارًا بسيطًا للأفكار لزيادة وضوح المغزى.

**هل القصة ملائمة للأطفال**: نعم / لا

<</SYS>>

 كان هناك أمير شاب قرر أن يغادر مملكته بحثًا عن الحكمة والمغامرة. في رحلته، مر بعدة ممالك، كل منها تمتلك أسرارًا خاصة وشخصيات غامضة. في مملكة من الممالك، وجد أن الناس يعيشون في عزلة تامة ولا يتحدثون إلى بعضهم البعض. استغرب الأمير هذا التصرف، وعندما استفسر عن السبب، قيل له إن هناك خلافات قديمة بين الأسر في تلك المملكة.

أدرك الأمير أن الشقاق بين الناس سببه سوء الفهم وغياب التفاهم، فقرر محاولة التوسط بينهم ليفهموا حاجاتهم المشتركة. نجح في جمع بعض الأسر وإعادة العلاقات بينهم، لكنه ترك المملكة وهو يدرك أن الأمر لم يكن سهلاً ويتطلب جهدًا طويلًا للتغيير.

المغزى: القصة تعلم عن أهمية التفاهم والإصلاح، ولكنها تحتوي على موضوعات تتطلب نقاشات حول العلاقات الإنسانية المعقدة وأهمية الحوار، وهي موضوعات تتناسب أكثر مع فئة عمرية أكبر [/INST]   1. **ملاءمة المحتوى للأطفال (3-5 سنوات)**:
   - درجة: 2/5 - [تحتوي القصة على موضوعات معقدة مثل الخلافات وسوء الفهم بين الناس، والتي قد لا تكون مناسبة للأطفال في هذه الفئة العمرية. يفضل تعديل القصة لتتناسب مع القيم الإسلامية والعربية المناسبة للأطفال.]

2. **اللغة والمصطلحات**:
   - درجة: 4/5 - [تستخدم القصة مصطلحات بسيطة وواضحة، لكن يمكن تحسين بعض المصطلحات لجعلها أكثر ملاءمة للأطفال.]

3. **هيكل النص**:
   - درجة: 3/5 - [تتضمن القصة بداية ووسط ونهاية، لكن يمكن تحسين تسلسل الأحداث لجعلها أكثر وضوحًا وسهولة في الفهم للأطفال.]

4. **الشخصيات**:
   - درجة: 3/5 - [تتميز الشخصيات بالبساطة والقرب من بيئة الطفل، لكن يمكن تحسين وصف الشخصيات لجعلها أكثر جاذبية وتفاعلية للأطفال.]

5. **الخيال والواقعية**:
   - درجة: 4/5 - [توازن القصة بين الخيال والواقعية بشكل جيد، لكن يمكن تحسين بعض المشاهد لجعلها أكثر ملاءمة للأطفال وتجنب إثارة المخاوف.]

6. **التكرار في الأحداث**:
   - درجة: 4/5 - [تحتوي القصة على تكرار بسيط في الأحداث والجمل الأساسية، مما يساعد الأطفال على الفهم والاستيعاب.]

7. **الطول والانتباه**:
   - درجة: 3/5 - [تكون القصة قصيرة نسبيًا، لكن يمكن تقسيمها إلى أجزاء أصغر للحفاظ على انتباه الطفل بشكل أفضل.]

8. **القيم الثقافية والدينية**:
   - درجة: 3/5 - [تعزز القصة بعض القيم الإسلامية والعربية بشكل مناسب للأطفال، لكن يمكن تحسين التركيز على الأخلاق والقيم المناسبة لهذه الفئة العمرية.]

9. **قابلية الكلمات للتصور**:
   - درجة: 4/5 - [تستخدم القصة أوصافًا بسيطة وسهلة التصور للأماكن والشخصيات، مما يجعلها أقرب إلى خيال الطفل.]

10. **الملاءمة العاطفية**:
   - درجة: 3/5 - [تتضمن القصة مشاعر وأحداث تتناسب مع قدرة الطفل على الفهم والتعامل مع النص، لكن يمكن تحسين بعض المشاهد لتجنب التأثير السلبي على مشاعر الطفل.]

**المجموع النهائي**: 26/50

**تعليق نهائي مختصر**: القصة تحتاج إلى تعديلات لجعلها أكثر ملاءمة للأطفال بعمر 3-5 سنوات. يجب تبسيط المصطلحات والأحداث لجعلها أكثر وضوحًا وسهولة في الفهم للأطفال. يمكن تحسين تسلسل الأحداث والتكرار لتعزيز فهم الطفل للقصة.

**هل القصة ملائمة للأطفال**: لا

يمكن تحسين القصة وتعديل بعض العناصر لجعلها ملائمة للأطفال في الفئة العمرية المستهدفة. يجب تبسيط المصطلحات والأحداث لجعلها أكثر وضوحًا وسهولة في الفهم للأطفال. يمكن تحسين تسلسل الأحداث والتكرار لتعزيز فهم الطفل للقصة.  </s><s> [INST] "يا صغيرتي أنا اريد كل شيئ يخصك"
قالها وعينيه أسودتا برغبة بتملكها وأقسم ان قبلها  لن يكون لغيره 
"يخصني ؟"نظرت اليه بخوف مدركة عدم قدرته اكثر على السيطرة بنفسه 
 
حاصرها بين يديه عند باب مكتبه واقترب من إذنها هامساً 
"انا اريد جسدك ،قلبك ،وروحك " [/INST]  أعتذر عن تقديم تقييم لهذه القصة، لكن يجب أن أوضح أن المحتوى الذي قدمته لا يتناسب مع معايير المحتوى المناسب للأطفال بعمر 3-5 سنوات. يجب أن تكون القصص المقدمة للأطفال خالية من المشاهد أو العبارات التي قد تسبب القلق أو الخوف أو الإرباك للأطفال.

إذا كنت ترغب في تعديل القصة لجعلها ملائمة للأطفال، يمكنني مساعدتك في إعادة صياغتها بطريقة أكثر ملاءمة ومناسبة للفئة العمرية المستهدفة. يرجى تقديم مزيد من التفاصيل حول القصة والأحداث التي ترغب في تضمينها، وسأعمل على تعديلها وفقًا للمعايير المطلوبة.  </s><s> [INST]"""


## Execution
Let us now use the defined Model object, pair it with the input, and generate the response to your question:


In [19]:
question = input("Question: ")
formattedQuestion = f"""<s> [INST] {question} [/INST]"""
prompt = f"""{prompt_input}{formattedQuestion}"""
generated_response = model.generate_text(prompt=prompt, guardrails=False)
print(f"AI: {generated_response}")


Question: كان هناك طفل اسمه "علي" يحلم بالذهاب في رحلة إلى الفضاء. وفي يوم من الأيام، قرر علي أن يسافر إلى الشمس ليستمتع بالدفء هناك. أخذ علي طائرة سريعة وانطلق نحو الشمس. بعد بضع ساعات، وصل علي إلى الشمس واستمتع بحرارتها اللطيفة وأشعتها الذهبية. لعب علي حول الشمس لبعض الوقت ثم عاد إلى الأرض وهو يشعر بالسعادة.
AI:  1. **ملاءمة المحتوى للأطفال (3-5 سنوات)**:
   - درجة: 4/5 - [تتحدث القصة عن حلم الطفل بالذهاب إلى الفضاء والاستمتاع بالدفء، مما يعتبر موضوعًا مناسبًا للأطفال في هذه الفئة العمرية.]

2. **اللغة والمصطلحات**:
   - درجة: 4/5 - [تستخدم القصة مصطلحات بسيطة وواضحة تتناسب مع مستوى فهم الأطفال.]

3. **هيكل النص**:
   - درجة: 4/5 - [تحتوي القصة على بداية ووسط ونهاية واضحة، وتسلسل الأحداث مناسب للأطفال.]

4. **الشخصيات**:
   - درجة: 4/5 - [تتميز الشخصيات بالبساطة والقرب من بيئة الطفل، مثل الطفل علي وحلمه بالذهاب إلى الفضاء.]

5. **الخيال والواقعية**:
   - درجة: 4/5 - [توازن القصة بين الخيال والواقعية بشكل جيد، حيث يحلم الطفل بالذهاب إلى الفضاء ويستمتع بالدفء هناك.]

6. **التكرار في ال

# Next steps
You successfully completed this notebook! You learned how to use
watsonx.ai inferencing SDK to generate response from the foundation model
based on the provided input, model id and model parameters. Check out the
official watsonx.ai site for more samples, tutorials, documentation, how-tos, and blog posts.

<a id="copyrights"></a>
### Copyrights

Licensed Materials - Copyright © 2023 IBM. This notebook and its source code are released under the terms of the ILAN License.
Use, duplication disclosure restricted by GSA ADP Schedule Contract with IBM Corp.

**Note:** The auto-generated notebooks are subject to the International License Agreement for Non-Warranted Programs (or equivalent) and License Information document for watsonx.ai Auto-generated Notebook (License Terms), such agreements located in the link below. Specifically, the Source Components and Sample Materials clause included in the License Information document for Watson Studio Auto-generated Notebook applies to the auto-generated notebooks.  

By downloading, copying, accessing, or otherwise using the materials, you agree to the <a href="https://www14.software.ibm.com/cgi-bin/weblap/lap.pl?li_formnum=L-AMCU-BYC7LF" target="_blank">License Terms</a>  